In [ ]:
# Import modules
import ee
import geemap
import geopandas as gpd

In [ ]:
# Initialize Earth Engine
try:
    ee.Initialize()
except ee.EEException:
    ee.Authenticate()

In [5]:
def get_vis_params(image, aoi):
    # reduceRegion to get the minimum avg rainfall in the AOI
    min_rainfall = image.reduceRegion(
        reducer=ee.Reducer.min(),
        geometry=aoi,
        scale=5566,  # CHIRPS dataset native resolution
        maxPixels=1e8
    )

    # reduceRegion to get the maximum avg rainfall in the AOI
    max_rainfall = image.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=aoi,
    scale=5566,  # CHIRPS dataset native resolution
    maxPixels=1e8
    )

    # Get the result and print the minimum value
    try:
        rainfall_min_max = [min_rainfall.getInfo()['precipitation'], 
                            max_rainfall.getInfo()['precipitation']]
        vis_params = {'palette': ['darkred', 'red', 
                          'white', 'lightblue', 'blue', 'darkblue'],
               'min': rainfall_min_max[0], 'max': rainfall_min_max[1]}
        
    except KeyError:
        rainfall_min_max = [min_rainfall.getInfo()['constant'], 
                            max_rainfall.getInfo()['constant']]
        vis_params = {'palette': ['darkred', 'yellow', 'green', 'darkgreen'],
                'min': rainfall_min_max[0], 'max': rainfall_min_max[1]}
    
    return vis_params

In [6]:
# Get images for rainy season (Nov to Mar) only
def get_rain_season_images(rainfall_dataset, start_year, end_year):
    # Create empty image collection
    merged_image_collection = ee.ImageCollection([])
    
    for year in range(start_year, end_year):
        # Nov to Dec of current year
        start1 = ee.Date(f'{year}-11-01')
        end1 = ee.Date(f'{year}-12-31')
        season_part1 = rainfall_dataset.filterDate(start1, end1)

        # Jan to Mar of following year
        start2 = ee.Date(f'{year + 1}-01-01')
        end2 = ee.Date(f'{year + 1}-03-31')
        season_part2 = rainfall_dataset.filterDate(start2, end2)

        # Merge image collection
        merged_year_collection = season_part1.merge(season_part2)
        merged_image_collection = merged_image_collection.merge(merged_year_collection)

    return merged_image_collection

In [ ]:
def get_avg_rainfall(start_year, end_year, aoi_ee):
    
    rainfall_dataset = ee.ImageCollection("UCSB-CHG/CHIRPS/PENTAD") \
            .select('precipitation') 
    
    no_years = end_year - start_year

    # Get images for rainfal season between selected dates
    seasonal_rainfall_dataset = get_rain_season_images(rainfall_dataset, start_year, end_year)

    # Clip images to AOI and compute the mean
    avg_rainfall = seasonal_rainfall_dataset.sum() \
                    .divide(no_years) \
                    .clip(aoi_ee)

    return avg_rainfall

In [16]:
def get_rainfall_suitability_image(crop, start_year, end_year, aoi_ee):

    rainfall_image = get_avg_rainfall(start_year, end_year, aoi_ee,)

    suitability_ranges = {
        'Soybean':    [(600, 1000, 3), (500, 600, 2), (1000, 1200, 2), (400, 500, 1), (1200, 1300, 1)],
        'Groundnut':  [(500, 1000, 3), (400, 500, 2), (1000, 1100, 2), (300, 400, 1), (1100, 1200, 1)],
        'Sugarbean':  [(500, 1000, 3), (400, 500, 2), (1000, 1100, 2), (300, 400, 1), (1100, 1200, 1)],
        'Navybean':   [(500, 1000, 3), (400, 500, 2), (1000, 1100, 2), (300, 400, 1), (1100, 1200, 1)],
        'Maize':      [(600, 1200, 3), (500, 600, 2), (1200, 1300, 2), (400, 500, 1), (1300, 1400, 1)],
        'Cowpea':     [(400, 800, 3), (300, 400, 2), (800, 900, 2), (250, 300, 1), (900, 1000, 1)],
        'Sunflower':  [(400, 800, 3), (300, 400, 2), (800, 900, 2), (250, 300, 1), (900, 1000, 1)],
    }

    crop = crop.strip().replace(" ", "").title()
    ranges = suitability_ranges[crop]

    # Default suitability of 0 (no suitability)
    suitability_image = ee.Image(0).clip(aoi_ee)

    # Apply suitability levels
    for (min_val, max_val, score) in ranges:
        mask = rainfall_image.gte(min_val).And(rainfall_image.lt(max_val))
        suitability_image = suitability_image.where(mask, score)

    return suitability_image


In [ ]:
# # Open source table as an ee
# # country_gpkg = r"data\zambia_boundary.gpkg"
# country_gpkg = r"C:\Users\jedid\Documents\Code\github\crop_soil_mcda\data\zambia_boundary.gpkg"
# country_gdf = gpd.read_file(country_gpkg)
# country_ee = geemap.gdf_to_ee(country_gdf)

# # Preview the data
# # country_gdf.head()

In [ ]:
# start_year, end_year = 2019, 2025
# crop = "Sunflower"

# #  Get average rainfall
# avg_rainfall = get_avg_rainfall(start_year, end_year, country_ee,)


# # Get ranked crop suitable average rainfall
# ranked_suitable_rainfall = get_rainfall_suitability_image(crop, start_year, end_year, country_ee)


In [ ]:
# # Rainfall visualisastion parameters
# # avg_rainfall_vis_params = get_vis_params(avg_rainfall, country_ee)
# # suitable_rainfall_vis_params = get_vis_params(suitable_rainfall, country_ee)
# ranked_rainfall_vis_params = get_vis_params(ranked_suitable_rainfall, country_ee)

In [ ]:
# # Visualise layers
# m = geemap.Map()
# m.zoom_to_gdf(country_gdf)

# # m.add_layer(country_ee, name= 'Zambia')
# # m.add_ee_layer(avg_rainfall, vis_params= avg_rainfall_vis_params, name= "Rainfall")
# # m.add_colorbar(avg_rainfall_vis_params, orientation='vertical',  
# #                label= "Avg Rainfall (mm)", layer_name= "Rainfall Suitability")


# # m.add_ee_layer(suitable_rainfall, vis_params= suitable_rainfall_vis_params, name= f"{crop} Suitable Areas")

# m.add_ee_layer(ranked_suitable_rainfall, vis_params= ranked_rainfall_vis_params, name= f"{crop} Suitable Areas")
# m.add_colorbar(ranked_rainfall_vis_params, orientation='vertical',  
#                label= "Rainfall Suitability", layer_name= f"{crop} Rainfall Suitability")
# m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…